In [313]:
import os
import csv
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq
import time
import random
import unicodedata
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import *
import string
import re
from math import *

In [5]:
path = os.getcwd()
movies = pd.DataFrame(pd.read_html(path + "\\movies-1.html")[0])
movies.drop('Id', inplace=True, axis = 1)

In [111]:
# for i in range(len(movies)):
#     try:
#         response = rq.get(movies.URL[i])
#     except rq.exceptions.RequestException as e:
#         print(e)
#         time.sleep(20*60 + 30)
#         response = rq.get(movies.URL[i])
#     soup = BeautifulSoup(response.text, 'html.parser')
#     f = open('article_'+str(i)+'.html','w')
#     f.write(str(soup))
#     f.close()
#     time.sleep(random.choice(range(1,6)))

In [6]:
path1 = path + '\\Articles'

In [198]:
for i in range(10000):
    article = open(path1+'\\article_'+str(i)+'.html', 'r')
    soup = BeautifulSoup(article, 'html.parser')
    d = {}
    try:
        for x in soup.find('table', class_="infobox vevent").find('th').find_all_next('th'):
            d[x.text] = unicodedata.normalize('NFKD',x.next_sibling.get_text(separator = '<br/>').replace('<br/>', ',').strip())
    except:
        pass
    title = str(soup.select('h1')[0].text)
    
    start = soup.find('p')
    intro = start.text.strip()
    while len(intro) == 0:
        start = start.find_next('p')
        intro = start.text.strip()
    for elem in start.next_siblings:
        if elem.name != 'p':
            break
        intro += elem.text.strip()   
          
    try:
        start = soup.find('h2').find_next('p')
        plot = start.text.strip()
        for elem in start.next_siblings:
            if elem.name != 'p':
                break
            plot += elem.text.strip()
        
    except:
        plot = "NA"
    

        
        
    try :
            director = d['Directed by']
    except: 
            director = "NA"
            
    try :
            producer = d['Produced by']
    except: 
            producer = "NA"
            
    try :
        
            writer = d["Written by"]
    except:
            writer = "NA"
    
    
    try :
            starring = d["Starring"].strip()
    
    except:
            starring = "NA"
            
    try :
        
            music = d["Music by"]
    except :
            music = "NA"
        
    try :
            release_date = d["Release date"]
            
    except :
        
            release_date = "NA"
            
    try :
            run_time = d["Running time"]
            
    except :
            run_time = "NA"
            
    try :
            country = d["Country"]
            
    except :
            
            country = "NA"
            
    try :
            language = d["Language"]
            
    except:
            language = "NA"
            
            
    try :
            budget = d["Budget"]
            
    except :
            budget = "NA"
            

    with open(path+"\\TSV\\article_" + str(i) + ".tsv", "w" ,encoding="utf-8") as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow([title, intro, plot, director, producer, writer , starring, music, release_date,run_time, country , language , budget])


In [16]:
path2 = path+'\\TSV\\'

In [35]:
stop_words = set(stopwords.words('english')) 
stemmer = PorterStemmer()

In [207]:
allwords = []
for i in range(0,10000):
    with open(path2+"article_" + str(i) + ".tsv", encoding = "utf-8") as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            if row :
                tsv = row
    text = ' '.join([tsv[1],tsv[2]])
    text = text.lower()
    
    words = word_tokenize(text) #devide the text into substrings
    filtered1 = [w for w in words if not w in stop_words] #remove stop words
    filtered2 = list(filter(lambda word: word not in string.punctuation, filtered1))
    filtered3 = []
    for word in filtered2:
        try:
            filtered3 += re.findall(r'\w+', word) 
        except:
            pass
    
    filtered3 = [stemmer.stem(w) for w in filtered3] #stemming
    filtered4 = [c.replace("''", "").replace("``", "") for c in filtered3 ] #removing useless '' and  `` characters
    filtered4 = [f for f in filtered4 if len(f)>1]
    
    with open(path + "\\WORDS\\final_" + str(i) + ".tsv", "w" ,encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(filtered4)
    allwords += filtered4


In [215]:
allwords = set(allwords)

In [10]:
path2 = path+'\\TSV\\'

In [194]:
f = open(path2+'article_0.tsv', encoding = 'utf8')

In [216]:
with open(path2+'article_9973.tsv', encoding = 'utf8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        if row :
            tsv = row


In [311]:
import re

d = {}

if tsv[6] == 'NA':
    d['Starring'] = '0'
else:
    d['Starring'] = str(len(tsv[6].split(',')))

try:
    d['Release Year'] = re.search(r'\d{4}', tsv[8]).group(0)
except:
    d['Release Year'] = '0'
    
try:
    d['Runtime']      = re.search(r'\d+.*',tsv[9]).group(0)
except:
    d['Runtime']    = '0'

#some movies have running time expressed in reels, and the conversion in minutes is not univoque, so we'll just ignore those info
if re.search(r'min', d['Runtime']):
    d['Runtime'] = re.search(r'\d+[\.|\,|:]*\d*', d['Runtime']).group(0)
    d['Runtime'] = re.search(r'\d+', d['Runtime']).group(0)
else:
    d['Runtime'] = 0
    
try:
    d['Budget']   = re.findall(r'\$.*', tsv[12])[0]
except:
    d['Budget']  = '0'

if re.search(r'mil', d['Budget']):
    d['Budget']  = str(int(float(re.search(r'\d+[\.|\,]*\d*', d['Budget']).group(0))*10**6))
elif re.search(r',', d['Budget']):
    d['Budget'] = d['Budget'].replace(',', '').replace('$', '')
else:
    d['Budget'] = d['Budget'].replace('.', '').replace('$', '')

d

{'Starring': '3', 'Release Year': '1982', 'Runtime': '84', 'Budget': '0'}

In [319]:
q = dict()
q['runtime'] = 25
Runtimes = [6,12,15,57,100,132]
minrun = min(Runtimes)
maxrun = max(Runtimes)
runscore = exp(-(int(q['runtime'] -int(d['Runtime']))**2)/100)
runscore

7.624459905389723e-16

In [2]:
q = dict()
oldnew = input("Do you prefer an old movie or a new-released movie? Please type O for Old and N for New:")
if oldnew == "O" :
    q["release"] = "O"
if oldnew == "N" :
    q["release"] = "N"
    
    
    
year = input("Do you want to specify the release year ?Please type Y for Yes and N for No: ")
if year == "N" :
    q["year"] = "NA"
if year == "Y" :
    year = input("please specify the release date") 
    q["year"] = year
    
    

Runtime = input("Do you want to specify the length of the movie? Please type Y for Yes and N for No:")
if Runtime == "N" :
    q["Runtime"] = "NA"
if Runtime == "Y" :
    Runtime = input("please specify the length of the movie")
    q["Runtime"] = Runtime
    
    
starring = input("Is number of stars an important factor for you? Please type Y for Yes and N for No:")
q["starring"] = starring


budget = input("Is movie budget an important factor for you? Please type Y for Yes and N for No:")
q["budget"] = budget


Do you prefer an old movie or a new-released movie? Please type O for Old and N for New:O
Do you want to specify the release year ?Please type Y for Yes and N for No: Y
please specify the release date1924
Do you want to specify the length of the movie? Please type Y for Yes and N for No:Y
please specify the length of the movie120
Is number of stars an important factor for you? Please type Y for Yes and N for No:Y
Is movie budget an important factor for you? Please type Y for Yes and N for No:10
